# Connecting to ADW via an SSH Tunnel

In [1]:
import os
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

In [2]:
PORT=5432
REMOTE_SSH_PORT=22
LOCAL_BIND_PORT=55009
PRIVATE_KEY_PATH='~/.ssh/id_rsa'
TUNNEL = SSHTunnelForwarder(
        (os.getenv('SSHPROXY'), REMOTE_SSH_PORT),
        ssh_username=os.getenv('ADWUSER'),
        ssh_private_key=PRIVATE_KEY_PATH,
        remote_bind_address=(os.getenv('ADWHOST'), PORT),
        local_bind_address=('localhost', LOCAL_BIND_PORT),
        threaded=False,
)

def query(sqlQuery):
    try:
        TUNNEL.close()
        TUNNEL.start()
        with psycopg2.connect(
            database=os.getenv('ADWNAME'),
            user=os.getenv('ADWUSER'),
            password=os.getenv('ADWPSWD'),
            host=TUNNEL.local_bind_host,
            port=TUNNEL.local_bind_port,
        ) as conn:
            with conn.cursor() as cur:
                cur.execute(sqlQuery)
                temporaryTable = cur.fetchall()
                dataz = pd.DataFrame.from_records(temporaryTable, coerce_float=True)
                colnames = [desc[0] for desc in cur.description]
                columnDict = dict([(i, colnames[i]) for i in range(len(colnames))])
                dataz = dataz.rename(columns=columnDict)
                print("Query returned", "{:,}".format(len(dataz)), "row(s)")
                return dataz
    except:
        TUNNEL.close()

In [3]:
sql = '''
SELECT
	a.id::text AS appid
	,a.created_at
	,a.retailer_id
	,orig.preapproval_id::text AS preapproval_id
	,email
	,CASE
		WHEN k._value LIKE $$%"AUTO": "D"%$$ THEN 'D'
		WHEN k._value LIKE $$%"AUTO": "A"%$$ THEN 'A'
		ELSE '?' END AS kount_decision
	,CASE WHEN orig.id IS NOT NULL THEN 1 ELSE 0 END AS originatd
FROM lms_public.lms_application a
LEFT JOIN adw.lms_reporting.origination orig
	ON a.id = orig.preapproval_id
LEFT JOIN lms_public.lms_applicationkeyval k
	ON a.id = k.container_id
	AND k.key = 'kount_json_response'
WHERE a.created_at >= '2019-12-16 14:05:00'
	--	AND a.retailer_id = 820
ORDER BY a.created_at DESC
;'''

In [4]:
df = query(sql)

Query returned 23,856 row(s)


In [5]:
df

,appid,created_at,retailer_id,preapproval_id,email,kount_decision,originatd
0,2648044,2019-12-19 21:33:57.413055+00:00,1033,None,maharperjaklin@yahoo.com,?,0
1,2648043,2019-12-19 21:33:15.052260+00:00,1137,None,fireguy741@gmail.com,?,0
2,2648042,2019-12-19 21:33:11.073939+00:00,1033,None,gschories90@gmail.com,?,0
3,2648041,2019-12-19 21:33:11.007664+00:00,1033,None,kingsvent45@yahoo.com,?,0
4,2648040,2019-12-19 21:33:01.116578+00:00,1033,None,kellyhorton80@yahoo.com,?,0
...,...,...,...,...,...,...,...
23851,2624268,2019-12-16 14:07:22.619178+00:00,1033,None,cassieboden@gmail.com,?,0
23852,2624267,2019-12-16 14:06:55.278167+00:00,1033,None,shanny2084@gmail.com,?,0
23853,2624266,2019-12-16 14:05:16.226841+00:00,1033,None,robin.joann@yahoo.com,?,0
23854,2624265,2019-12-16 14:05:15.925593+00:00,1001,None,tichelle1@yahoo.com,?,0
